In [ ]:
'''_In this script, we ..._'''

In [ ]:
# -*- coding: utf-8 -*- #
# Authors: gracecaj@stud.ntnu.no, sbaarak@stud.ntnu.no, & annakale@stud.ntnu.no
# Date: 19/09/2023

In [ ]:
#%%
#* Import the code needed
import os
from pathlib import Path
import numpy as np
import pandas as pd

In [ ]:
#%%
#* Define paths
directory_path = Path('../../')

# Import data needed (pop, ppd, type split)

In [ ]:
pop = pd.read_excel(
    directory_path / 'data' / 'regression_data' / 'Population' / 'regression_Population.xlsx',
    index_col=0)

PpD = pd.read_excel(
    directory_path / 'data' / 'regression_data' / 'PpD' / 'regression_PpD.xlsx',
    index_col=0)

#! Need to have type split ready in excel form
TS_cj = pd.read_excel(
    directory_path / 'data' / '__' / '__' / '__.xlsx',
    index_col=0)

lifetime = pd.read_excel(
    directory_path / 'data' / 'regression_data' / 'Dwelling_Lifetime' / 'Dwelling_Lifetime_Final_Data.xlsx',
    index_col=0, usecols = 'lifetime')


#! I don't think this value is actually standard deviation, but something else. Double check Bergsdal paper
lifetime_standard_deviation = pd.read_excel(
    directory_path / 'data' / 'regression_data' / 'Dwellling_Lifetime' / 'Dwelling_Lifetime_Final_Data.xlsx',
    index_col=0, usecols = 'std_dev')


# Import DSM model


In [ ]:
# getting the name of the directory where the this file is present.
current = os.path.dirname(os.path.realpath('this'))
 
# Getting the parent directory name where the current directory is present.
parent = os.path.dirname(current)
#make the UA path
module_path = parent + '/modules'
#append path to options to import from
sys.path.append(module_path)

import dynamic_stock_model as dsm

# Use pop and PpD to calculate stock over time

In [ ]:
stock_t = np.multiply(pop,PpD)

stock_t_SFH = np.multiply(stock_t,TS_cj['SFH'])
stock_t_TH = np.multiply(stock_t,TS_cj['TH'])
stock_t_AB = np.multiply(stock_t,TS_cj['AB'])

# Enter stock into stock driven model to get inflows by cohort over time

In [ ]:
from dynamic_stock_model import DynamicStockModel

#! The following function and runs of it should effectively iterate things over for each of the types

def stock_model(stock):
    dwellings_stock_driven = DynamicStockModel(t=np.arange(1600, 2050),
                         s=stock,
                         lt={'Type': 'Normal', 
                             'Mean': [lifetime],
                             'StdDev':[lifetime_standard_deviation] 
                             }
                        )
    inflow_tc = dwellings_stock_driven.i_c
     
    inflow_tcj = np.multiply(inflow_tc,TS_cj)
    
    dwellings_inflow_driven = DynamicStockModel(t=np.arange(1600, 2050),
                        i=inflow_tcj,
                        lt={'Type': 'Normal', 
                            'Mean': np.array([lifetime]),
                            'StdDev': np.array([lifetime_standard_deviation]) 
                            }
                        )
    dwellings_inflow_driven.compute_s_c_inflow_driven()
    return dwellings_inflow_driven.s_c

s_c_SFH = stock_model(stock_t_SFH)
s_c_TH = stock_model(stock_t_TH)
s_c_AB = stock_model(stock_t_AB)

#! make some kind of stacked line chart with this. also compute sum

s_c_total = s_c_SFH + s_c_TH + s_c_AB




#! I'm not sure if we can use a vector of different lifetime values or if there can only be one constant. Check DSM module

dwellings_stock_driven = DynamicStockModel(t=np.arange(1600, 2050),
                         s=stock_t,
                         lt={'Type': 'Normal', 
                             'Mean': [lifetime],
                             'StdDev':[lifetime_standard_deviation] 
                             }
                        )

#! I think the following will work but I'm not 100% sure
# Calculate the inflow
inflow_tc = dwellings_stock_driven.i_c

# multiply inflow by type split to get inflow by type split and cohort

In [ ]:
inflow_tcj = np.multiply(inflow_tc,TS_cj)

# enter inflow by type split and cohort into inflow driven model to get stock by cohort and type split over time

In [ ]:
# Define our dynamic stock model using our data and parameters
dwellings_inflow_driven = DynamicStockModel(t=np.arange(1600, 2050),
                        i=inflow_tcj,
                        lt={'Type': 'Normal', 
                            'Mean': np.array([lifetime]),
                            'StdDev': np.array([lifetime_standard_deviation]) 
                            }
                        )


#! I think the following will work but I'm not 100% sure
# Calculate the stock by cohort
dwellings_inflow_driven.compute_s_c_inflow_driven()
s_c = dwellings_inflow_driven.s_c
